## Agents

Agents use an LLM to determine which actions to perform and in what order. An action can be either using a tool and oberving its output of returning it to the user. To use an agent. In addition to the concept of an LLM, it is important to understand a new concept and that of a "tool".

In [11]:
import os
import openai 
import pickle
from dotenv import load_dotenv, find_dotenv
from langchain import PromptTemplate, OpenAI, LLMChain

load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")

In [12]:
with open("vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f) 

## Tools
Functions that agents can be interact with the world. Common utilities such as search, other chains or even other agents.

In [3]:
from langchain.agents import load_tools
from langchain.llms import OpenAI

llm = OpenAI(model_name='text-davinci-003',
             temperature=0.7,
             openai_api_key=api_key)

In [5]:
tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method Chain.run of LLMMathChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion:

In [6]:
from langchain.agents import Tool
tool_list = [
    Tool(
        name="Math Tool",
        func=tools[0].run,
        description="Tools to calculate, noting else"
    )
]

In [7]:
from langchain.agents import initialize_agent

agent = initialize_agent(tool_list,
                         llm,
                         agents="zero-shot-react-description",
                         verbose=True)
agent.run("How are you?")



> Entering new  chain...
 This is not a math question.
Action: No action necessary.
Action Input: N/A
Observation: No action necessary. is not a valid tool, try another one.
Thought: This is not a math problem that can be solved.
Final Answer: I'm doing well, thank you!

> Finished chain.


"I'm doing well, thank you!"

In [9]:
agent.run("What is 12344556 multiply by 20?")



> Entering new  chain...


 I need to use the Math Tool to solve this.
Action: Math Tool
Action Input: 12344556 * 20
Observation: Answer: 246891120
Thought: I now know the final answer
Final Answer: 246891120

> Finished chain.


'246891120'

## Custom Tools

You can also create your own tools by creating a class that inherits from BaseTool

In [15]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class CustomSearchTool(BaseTool):
    name = "restaurant search"
    description = "useful for when you need to answer questions about our restaurant."
    
    def _run(self, query:str, run_manager: Optional[CallbackManagerForToolRun]=None)->str:
        store = vectorstore.as_retriever()
        docs= store.get_relevant_documents(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)
    
    async def _arun(self, query:str, run_manager: Optional[AsyncCallbackManagerForToolRun]=None)->str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [16]:
from langchain.agents import AgentType

tools = [CustomSearchTool()]
agent = initialize_agent(tools, llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [17]:
agent.run("when does the restaurant open?")



> Entering new  chain...
 I need to find out the opening hours
Action: restaurant search
Action Input: opening hours
Observation: Q: What are the hours of operation for your restaurant?
A: Our restaurant is open from 11 a.m. to 10 p.m. from Monday to Saturday. On Sundays, we open at 12 p.m. and close at 9 p.m.

Q: What type of cuisine does your restaurant serve?
A: Our restaurant specializes in contemporary American cuisine with an emphasis on local and sustainable ingredients.
Q: Do you have any specials or promotions running currently?
A: Yes, we have a Happy Hour promotion from 3 to 5 p.m. on weekdays, with special prices on selected appetizers and drinks.
Q: Do you offer vegetarian or vegan options?
A: Yes, we have a range of dishes to cater to vegetarians and vegans. Please let our staff know about any dietary restrictions you have when you order.
Q: What steps is your restaurant taking to ensure safety amid the ongoing pandemic?
A: We adhere to strict health and safety protocol

'Our restaurant is open from 11 a.m. to 10 p.m. from Monday to Saturday and from 12 p.m. to 9 p.m. on Sundays.'